In [2]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout
from tensorflow.keras.models import Sequential,load_model
import cv2
import numpy as np
from keras.utils import to_categorical
import glob
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [3]:
def preprocessing(filepath):
    data=[cv2.imread(i) for i in filepath]
    data=[cv2.resize(i,(150,150)) for i in data]
    data=np.array(data)
    return data/255.0

In [4]:
no_class_filepath=glob.glob(r"Brain MRI Images for Brain Tumor Detection\no\*")
yes_class_filepath=glob.glob(r"Brain MRI Images for Brain Tumor Detection\yes\*")
x_no=preprocessing(no_class_filepath)
x_yes=preprocessing(yes_class_filepath)
y=[0 for i in range(len(no_class_filepath))]+[1 for i in range(len(yes_class_filepath))]
y=to_categorical(y)
x=np.concatenate((x_no,x_yes))

In [6]:
x.shape

(253, 150, 150, 3)

In [7]:
model=Sequential()
model.add(Conv2D(64,(4,4),input_shape=(150,150,3),activation="relu",padding="SAME"))
model.add(MaxPooling2D((3,3)))
model.add(Conv2D(64,(3,3),input_shape=(150,150,3),activation="relu",padding="SAME"))
model.add(MaxPooling2D((3,3)))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(2,activation="sigmoid"))

In [8]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [10]:
model.fit(x,y,validation_split=0.2,epochs=10)

Epoch 1/10
7/7 [==============================] - 1s 203ms/step - loss: 0.2349 - accuracy: 0.9010 - val_loss: 0.2341 - val_accuracy: 0.9216
Epoch 2/10
7/7 [==============================] - 1s 189ms/step - loss: 0.2415 - accuracy: 0.9010 - val_loss: 0.7090 - val_accuracy: 0.6471
Epoch 3/10
7/7 [==============================] - 1s 198ms/step - loss: 0.1832 - accuracy: 0.9257 - val_loss: 0.3419 - val_accuracy: 0.9216
Epoch 4/10
7/7 [==============================] - 1s 196ms/step - loss: 0.1639 - accuracy: 0.9257 - val_loss: 0.4354 - val_accuracy: 0.8627
Epoch 5/10
7/7 [==============================] - 1s 200ms/step - loss: 0.1087 - accuracy: 0.9802 - val_loss: 0.3800 - val_accuracy: 0.8824
Epoch 6/10
7/7 [==============================] - 1s 196ms/step - loss: 0.0722 - accuracy: 0.9802 - val_loss: 0.3294 - val_accuracy: 0.9020
Epoch 7/10
7/7 [==============================] - 1s 191ms/step - loss: 0.0599 - accuracy: 0.9802 - val_loss: 0.5941 - val_accuracy: 0.8627
Epoch 8/10
7/7 [====

In [11]:
model.save("brain_tumour.h5")

In [12]:
filepath=r"Brain MRI Images for Brain Tumor Detection\\no\\1 no.jpeg"

In [13]:
from keras.models import load_model
import cv2
import numpy as np
def preprocessing(i):
    data=cv2.imread(i)
    data=cv2.resize(data,(150,150))
    data=np.array(data)
    data=data.reshape(-1,150,150,3)
    return data/255.0

model=load_model("brain_tumour.h5")
data=preprocessing(filepath)
s=model.predict_proba(data)
p=model.predict_classes(data)[0]

if p==0:
    print("No tumour detected and the probability of no tumour is :",s[0][0])
else :
    print("Tumour is detected and probability is of having tumour is :",s[0][1])

Instructions for updating:
Please use `model.predict()` instead.
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
No tumour detected and the probability of no tumour is : 0.9998872
